In [6]:
import math

def calculate_trading_strategy_improved(buy_price, budget, min_step,
                                        buy_fee_percent, sell_fee_percent,
                                        profit_percent):
    """
    Расширенная функция расчёта сделки с учётом нескольких важных моментов:

    1) При продаже комиссия (sell_fee_percent) вычитается из самой монеты:
       Чтобы продать ровно X (кратное min_step), нужно купить чуть больше X,
       т.к. часть уйдёт на комиссию биржи в монете.

    2) При покупке комиссия (buy_fee_percent) вычитается из USDT:
       Нужно закладывать «лишние» USDT, чтобы после комиссии мы действительно
       получили нужное количество монет.

    3) Нужна кратность min_step:
       Чтобы избежать «мусорных» остатков (ниже минимального шага),
       итоговое количество монеты при продаже должно быть ровно X = N * min_step.

    4) Учитываем бюджет (budget):
       Смотрим, сколько максимально монет X мы можем позволить себе купить,
       чтобы стоимость + комиссия покупки не вышла за этот budget.
       При этом X – это объём, который реально **продадим** (уже без мусора).

    5) Цену продажи считаем с желаемым профитом (profit_percent).
       Финальная выручка = X * sell_price.
       Но X – это именно то количество, которое попадёт в продажу после вычета
       комиссии в монете.
    """

    # -----------------------------
    # ВСПОМОГАТЕЛЬНЫЕ ПЕРЕМЕННЫЕ
    # -----------------------------

    # Коэффициент, учитывающий комиссию при покупке в USDT:
    # Т.е. покупая монету по buy_price, придётся заплатить чуть дороже (buy_fee_percent).
    # Фактическая "эффективная" цена = buy_price * (1 + buy_fee_percent/100)
    buy_price_with_fee_factor = buy_price * (1 + buy_fee_percent / 100)

    # Коэффициент, учитывающий комиссию при продаже в монете:
    # Если мы держим total_coins монеты, то при продаже биржа снимет часть
    # total_coins * sell_fee_percent/100, и на рынок пойдёт только
    # total_coins * (1 - sell_fee_percent/100).
    # Чтобы продать ровно X монет, нам нужно иметь X / (1 - sell_fee_percent/100).
    coin_after_sell_fee_factor = (1 - sell_fee_percent / 100)

    # Целевую цену продажи возьмём как "цена покупки + желаемая прибыль".
    # Например, если profit_percent=2, то хотим на 2% выше buy_price.
    # (Иногда сюда добавляют ещё учёт sell_fee_percent, но мы сейчас заложим комиссию
    #  в объёмах, а не в самой цене).
    sell_price = buy_price * (1 + profit_percent / 100)

    # -----------------------------
    # РАСЧЁТ МАКСИМАЛЬНОГО X
    # -----------------------------
    # Пусть X — кол-во монет, которое мы хотим продать (после комиссий),
    # причём X должно быть кратно min_step.
    #
    # Чтобы реально иметь X на продажу, мы должны купить:
    #   total_coins = X / (1 - sell_fee_percent/100)
    #
    # А сколько это будет стоить в USDT, с учётом комиссии покупки?
    #   cost_in_usdt = total_coins * buy_price_with_fee_factor
    #                = X / coin_after_sell_fee_factor * (buy_price * (1 + buy_fee_percent/100))
    #
    # Мы не можем превысить наш budget (т.е. cost_in_usdt <= budget).
    #
    # => X / coin_after_sell_fee_factor * buy_price_with_fee_factor <= budget
    # => X <= budget * (coin_after_sell_fee_factor / buy_price_with_fee_factor)
    #

    if coin_after_sell_fee_factor <= 0:
        # Если sell_fee_percent >= 100%, торговать бессмысленно.
        return {
            "max_sellable_amount": 0,
            "total_coins_to_buy": 0,
            "total_usdt_needed": 0,
            "sell_price": 0,
            "final_revenue": 0,
            "net_profit": 0
        }

    # "сырое" максимальное X до округления
    raw_max_x = budget * (coin_after_sell_fee_factor / buy_price_with_fee_factor)

    # Функция для округления вниз до шага min_step
    def floor_to_step(value, step):
        return math.floor(value / step) * step

    # Округлим X до ближайшего снизу целого кратного min_step
    X_adjusted = floor_to_step(raw_max_x, min_step)

    # Если X_adjusted == 0, значит даже минимальное количество (min_step) купить не получается
    if X_adjusted <= 0:
        return {
            "max_sellable_amount": 0,
            "total_coins_to_buy": 0,
            "total_usdt_needed": 0,
            "sell_price": sell_price,
            "final_revenue": 0,
            "net_profit": 0
        }

    # Теперь считаем, сколько реально монет нужно купить, чтобы после продажи осталось X_adjusted
    total_coins_needed = X_adjusted / coin_after_sell_fee_factor  # "резерв" до комиссии

    # Общая сумма USDT, которую мы потратим на покупку (включая комиссию покупки)
    total_usdt_needed = total_coins_needed * buy_price_with_fee_factor

    # Итого после продажи мы получим X_adjusted * sell_price
    final_revenue = X_adjusted * sell_price

    # Чистая прибыль = финальная выручка - то, что мы потратили
    net_profit = final_revenue - total_usdt_needed

    return {
        # Кол-во монет, которое реально удастся продать без остатков
        "max_sellable_amount": X_adjusted,
        
        # Сколько монет придётся купить (учитывая, что часть "сгорит" на sell_fee)
        "total_coins_to_buy": total_coins_needed,
        
        # Сколько USDT реально уйдёт на покупку + комиссию покупки
        "total_usdt_needed": total_usdt_needed,
        
        # Цена, по которой мы продаём (без ручного добавления sell_fee; 
        # комиссию при продаже учитываем объёмом монет)
        "sell_price": sell_price,
        
        # Итоговая выручка от продажи X_adjusted
        "final_revenue": final_revenue,
        
        # Чистая прибыль после всех операций
        "net_profit": net_profit
    }


# -----------------------------
# ПРИМЕР ИСПОЛЬЗОВАНИЯ:
# -----------------------------

if __name__ == "__main__":
    buy_price = 0.12922   # Цена покупки за 1 BTC
    budget = 10           # Бюджет в USDT
    min_step = 0.1     # Минимальный лот для BTC
    buy_fee_percent = 0.1 # Комиссия при покупке (%)
    sell_fee_percent = 0.1# Комиссия при продаже (%)
    profit_percent = 2    # Желаемая прибыль (%)

    result = calculate_trading_strategy_improved(
        buy_price, budget, min_step,
        buy_fee_percent, sell_fee_percent,
        profit_percent
    )

    print(result)


{'max_sellable_amount': 77.2, 'total_coins_to_buy': 77.27727727727728, 'total_usdt_needed': 9.99575553953954, 'sell_price': 0.13180440000000002, 'final_revenue': 10.175299680000002, 'net_profit': 0.17954414046046274}
